In [1]:
import os
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("darkgrid")

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [3]:
import mlflow

**Download the data**

- [NYC taxi data](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page)
    - 2022, Yellow, January
    - 2022, Yellow, February

In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])
    df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])

    # create duration-column
    df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df["duration"] = df["duration"].dt.total_seconds() / 60

    # remove outliers
    df = df[(df["duration"] >= 1) & (df["duration"] <= 60)].copy()

    # convert categorical features to strings (to get 1-hot encoding)
    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df

train_file = "data/yellow_tripdata_2022-01.parquet"
val_file = "data/yellow_tripdata_2022-02.parquet"

df_train = read_dataframe(train_file)
print("Train: ", df_train.shape)

df_val = read_dataframe(val_file)
print("Val: ", df_val.shape)

Train:  (2421440, 20)
Val:  (2918187, 20)


**Create new features**

In [9]:
df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [10]:
categorical = ["PU_DO"]
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [11]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

**Linear Regression**

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

5.530168304657237

In [14]:
models_dir = "./models/"
if not os.path.exists(models_dir):
    os.mkdir(models_dir)

with open("./models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

## Experiment Tracking with MLflow

In [21]:
# mlflow.set_tracking_uri(remote_server_uri)

with mlflow.start_run():

    mlflow.set_tag("developer", "johannes")

    mlflow.log_param("train-data-path", "./data/yellow_tripdata_2022-01.parquet")
    mlflow.log_param("valid-data-path", "./data/yellow_tripdata_2022-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

MlflowException: The configured tracking uri scheme: 'file' is invalid for use with the proxy mlflow-artifact scheme. The allowed tracking schemes are: {'http', 'https'}